# Исследование данных об инвестиции венчурных фондов в компании-стартапы

### Описание проекта

Проект выполнен на основе базы данных, которая хранит информацию о венчурных фондах и инвестициях в компании-стартапы. Эта база данных основана на датасете Startup Investments, опубликованном на популярной платформе для соревнований по исследованию данных Kaggle. 

**Задача**: произвести различные выгрузки данных венчурных фондов с помощью SQL.

#### Описание данных

- `acquisition` — cодержит информацию о покупках одних компаний другими.
- `company` — cодержит информацию о компаниях-стартапах.
- `education` — хранит информацию об уровне образования сотрудников компаний.
- `fund` — хранит информацию о венчурных фондах. 
- `funding_round` — содержит информацию о раундах инвестиций. 
- `investment` — содержит информацию об инвестициях венчурных фондов в компании-стартапы.
- `people` — содержит информацию о сотрудниках компаний-стартапов.

### Написание запросов

1. Все записи из таблицы `company по компаниям`, которые закрылись.

In [ ]:
SELECT COUNT(*) AS closed_companies_count
FROM company
WHERE status = 'closed';

2. Количество привлечённых средств для новостных компаний США из таблицы `company` с сортировкой значений по убыванию поля `funding_total`.

In [ ]:
SELECT funding_total
FROM company
WHERE category_code = 'news' AND country_code = 'USA'
ORDER BY funding_total DESC;

3. Общая сумма сделок по покупке одних компаний другими в долларах. Сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

In [ ]:
SELECT SUM(price_amount) AS total_deal_amount
FROM acquisition
WHERE term_code = 'cash' AND acquired_at >= '2011-01-01' AND acquired_at <= '2013-12-31';

4. Имя, фамилия и название аккаунтов людей в поле `network_username`, у которых названия аккаунтов начинаются на 'Silver'.

In [ ]:
SELECT first_name, last_name, twitter_username
FROM people
WHERE twitter_username LIKE 'Silver%';

5. Информация о людях, у которых названия аккаунтов в поле `network_username` содержат подстроку 'money', а фамилия начинается на 'K'.

In [ ]:
SELECT *
FROM people
WHERE twitter_username LIKE '%money%' AND last_name LIKE 'K%';

6. Общая сумма привлечённых инвестиций для каждой страны, которые получили компании, зарегистрированные в этой стране с сортировкой по убыванию этой суммы. 

In [ ]:
SELECT country_code, SUM(funding_total) AS total_investment
FROM company
GROUP BY country_code
ORDER BY total_investment DESC;

7. Составление таблицы, в которую входит дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату. Оставлены только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

In [ ]:
SELECT funded_at, MIN(raised_amount) AS min_investment, MAX(raised_amount) AS max_investment
FROM funding_round
GROUP BY funded_at
HAVING MIN(raised_amount) <> 0 AND MIN(raised_amount) <> MAX(raised_amount);

8. Создание поля с категориями:
- Для фондов, которые инвестируют в 100 и более компаний, назначена категория `high_activity`.
- Для фондов, которые инвестируют в 20 и более компаний до 100 — `middle_activity`.
- Если количество инвестируемых компаний фонда не достигает 20 — `low_activity`.


Отображены все поля таблицы `fund` и новое поле с категориями.

In [ ]:
SELECT *,
       CASE
           WHEN invested_companies >= 100 THEN 'high_activity'
           WHEN invested_companies >= 20 AND invested_companies < 100 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS category
FROM fund;

9. Расчет округлённого до ближайшего целого числа среднего количества инвестиционных раундов, в которых фонд принимал участие, для каждой категории. Выведены категория и среднее число инвестиционных раундов с сортировкой по возрастанию.

In [ ]:
SELECT category, ROUND(AVG(investment_rounds)) AS average_rounds
FROM (
    SELECT *,
           CASE
               WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
           END AS category
    FROM fund
) AS subquery
GROUP BY category
ORDER BY average_rounds ASC;

10. Анализ стран, в которых находятся фонды, чаще всего инвестирующие в стартапы. Для каждой страны рассчитано минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключены страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. Выгружены десять самых активных стран-инвесторов с сортировкой по среднему количеству компаний по убыванию и с сортировкой по коду страны в лексикографическом порядке.

In [ ]:
SELECT
  country_code,
  MIN(invested_companies) AS min_companies,
  MAX(invested_companies) AS max_companies,
  AVG(invested_companies) AS avg_companies
FROM
  fund
WHERE
  EXTRACT(YEAR FROM founded_at) BETWEEN 2010 AND 2012
GROUP BY
  country_code
HAVING
  MIN(invested_companies) > 0
ORDER BY
  avg_companies DESC, country_code ASC
LIMIT 10;

11. Имена и фамилии всех сотрудников стартапов с информацией об учебном заведении, которое окончил сотрудник.

In [ ]:
SELECT
  p.first_name,
  p.last_name,
  e.instituition AS educational_institution
FROM
  people AS p
LEFT JOIN
  education AS e ON p.id = e.person_id;

12. Количество учебных заведений, которые окончили сотрудники компаний. Выведен топ-5 компаний по количеству университетов.

In [ ]:
SELECT
  c.name AS company_name,
  COUNT(DISTINCT e.instituition) AS num_universities
FROM
  company AS c
JOIN
  people AS p ON c.id = p.company_id
JOIN
  education AS e ON p.id = e.person_id
GROUP BY
  c.name
ORDER BY
  num_universities DESC
LIMIT 5;

13. Список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

In [ ]:
SELECT DISTINCT name
FROM company
WHERE status = 'closed'
      AND id IN (SELECT company_id
                 FROM funding_round
                 WHERE is_first_round = 1
                 AND is_last_round = 1)

14. Список уникальных номеров сотрудников, которые работают в компаниях, отобранных на предыдущем шаге.

In [ ]:
SELECT DISTINCT p.id
FROM people AS p
JOIN company AS c ON p.company_id = c.id
WHERE c.id IN (
  SELECT company_id
  FROM funding_round
  WHERE is_first_round = 1
    AND is_last_round = 1
    AND company_id IN (
      SELECT id
      FROM company
      WHERE status = 'closed'
    )
);

15. Составление таблицы с уникальными парами с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

In [ ]:
SELECT DISTINCT p.id AS person_id, e.instituition
FROM people AS p
JOIN company AS c ON p.company_id = c.id
JOIN education AS e ON p.id = e.person_id
WHERE c.id IN (
  SELECT company_id
  FROM funding_round
  WHERE is_first_round = 1
    AND is_last_round = 1
    AND company_id IN (
      SELECT id
      FROM company
      WHERE status = 'closed'
    )
);

16. Количество учебных заведений для каждого сотрудника из предыдущего шага. При подсчёте учитывается, что некоторые сотрудники могли окончить одно и то же заведение дважды.

In [ ]:
SELECT p.id AS person_id, COUNT(e.instituition) AS num_institutions
FROM people AS p 
JOIN company AS c ON p.company_id = c.id
JOIN education AS e ON p.id = e.person_id
WHERE c.id IN (
    SELECT company_id
    FROM funding_round
    WHERE is_first_round = 1
        AND is_last_round = 1
        AND company_id IN (
            SELECT id
            FROM company 
            WHERE status = 'closed'
            )
)
GROUP BY p.id;

17. Дополнение предыдущего запроса: среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний.

In [ ]:
SELECT AVG(num_institutions) AS average_institutions
FROM (SELECT p.id AS person_id, COUNT(e.instituition) AS num_institutions
FROM people AS p 
JOIN company AS c ON p.company_id = c.id
JOIN education AS e ON p.id = e.person_id
WHERE c.id IN (
    SELECT company_id
    FROM funding_round
    WHERE is_first_round = 1
        AND is_last_round = 1
        AND company_id IN (
            SELECT id
            FROM company 
            WHERE status = 'closed'
            )
)
GROUP BY p.id) AS subquery;

18. Среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Socialnet.

In [ ]:
SELECT AVG(num_institutions) AS average_num_institutions
FROM (
    SELECT p.id AS person_id, COUNT(e.instituition) AS num_institutions
    FROM people AS p
    JOIN company AS c ON p.company_id = c.id
    JOIN education AS e ON p.id = e.person_id
    WHERE c.name = 'Socialnet'
    GROUP BY p.id
) AS subquery;

19. Составление таблицы из полей:
- `name_of_fund` — название фонда;
- `name_of_company` — название компании;
- `amount` — сумма инвестиций, которую привлекла компания в раунде.
В таблицу вошли данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

In [ ]:
SELECT DISTINCT f.name AS name_of_fund,
c.name AS name_of_company,
fr.raised_amount AS amount 
FROM company AS c 
LEFT OUTER JOIN investment AS i ON c.id = i.company_id
RIGHT OUTER JOIN fund AS f ON i.fund_id = f.id 
LEFT OUTER JOIN funding_round AS fr ON i.funding_round_id = fr.id
WHERE c.milestones > 6 
AND EXTRACT(YEAR FROM CAST(funded_at AS date)) BETWEEN '2012' AND '2013'; 

20. Выгрузка таблицы со следующими полями:
- название компании-покупателя;
- сумма сделки;
- название компании, которую купили;
- сумма инвестиций, вложенных в купленную компанию;
- доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.

Не учитываются те сделки, в которых сумма покупки равна нулю. Сортировка по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Выведены 10 записей.

In [ ]:
SELECT acq_comp.name AS acquiring_company,
       acq.price_amount,
       acquired_comp.name AS acquired_company,
       acquired_comp.funding_total,
       ROUND(acq.price_amount / acquired_comp.funding_total) AS ratio
FROM acquisition AS acq
JOIN company AS acq_comp ON acq.acquiring_company_id = acq_comp.id
JOIN company AS acquired_comp ON acq.acquired_company_id = acquired_comp.id
WHERE acq.price_amount > 0
  AND acquired_comp.funding_total > 0
ORDER BY acq.price_amount DESC, acquired_comp.name
LIMIT 10;

21. Выгрузка таблицы с названиями компаний из категории 'social', получившие финансирование с 2010 по 2013 год включительно и номером месяца, в котором проходил раунд финансирования.

In [ ]:
SELECT c.name AS company_name,
       EXTRACT(MONTH FROM fr.funded_at) AS funding_month
FROM company AS c
JOIN funding_round AS fr ON c.id = fr.company_id
WHERE c.category_code = 'social'
  AND fr.funded_at >= '2010-01-01' AND fr.funded_at <= '2013-12-31'
  AND fr.raised_amount != 0
ORDER BY c.name, fr.funded_at;

22. Данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды, сгруппированные по номеру месяца, включающие поля: 
- номер месяца, в котором проходили раунды;
- количество уникальных названий фондов из США, которые инвестировали в этом месяце;
- количество компаний, купленных за этот месяц;
- общая сумма сделок по покупкам в этом месяце.

In [ ]:
WITH
p1 AS (
    SELECT *
    FROM funding_round
    WHERE EXTRACT(YEAR FROM funded_at) BETWEEN '2010' AND '2013'
    ),
 
   
p2 AS (
    SELECT EXTRACT(MONTH FROM funded_at) AS month_round,
           COUNT(DISTINCT fund_id) AS count_fund
    FROM investment INNER JOIN p1 ON investment.funding_round_id = p1.id
    WHERE fund_id IN
        (SELECT id AS fund_id
        FROM fund
        WHERE country_code = 'USA')
    GROUP BY month_round
),
 
p3 AS (
    SELECT EXTRACT(MONTH FROM acquired_at) AS month_acquired,
           COUNT(acquired_company_id) AS count_company,
           SUM(price_amount) AS total
    FROM acquisition
    WHERE EXTRACT(YEAR FROM acquired_at) BETWEEN '2010' AND '2013'
    GROUP BY month_acquired
    )
 
SELECT month_round,
       count_fund,
       count_company,
       total
FROM p2 INNER JOIN p3 ON p2.month_round = p3.month_acquired 

23. Средняя сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах с сортировкой по среднему значению инвестиций за 2011 год от большего к меньшему.

In [ ]:
WITH
inv_2011 AS
(SELECT country_code,
       AVG(funding_total) AS average_2011
FROM company
WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2011
GROUP BY country_code),
         
inv_2012 AS
(SELECT country_code,
       AVG(funding_total) AS average_2012
FROM company
WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2012
GROUP BY country_code),
         
inv_2013 AS (
SELECT country_code,
       AVG(funding_total) AS average_2013
FROM company
WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2013
GROUP BY country_code)
 
SELECT inv_2011.country_code,
       average_2011,
       average_2012,
       average_2013
FROM inv_2011
INNER JOIN inv_2012 ON inv_2011.country_code = inv_2012.country_code
INNER JOIN inv_2013 ON inv_2012.country_code = inv_2013.country_code
ORDER BY average_2011 DESC;